# Indicator optimization
---
This notebook aims to optimize indicator parameters for optimal buy and sell strategies for groups of similar stocks. Group of stocks is analyzed by modified function ```test_strategy()``` from *multiple_analysis_.ipynb* notebook. Main modification to the ```test_strategy()``` is new input, ```pd.Series```, which holds parameters used by specific buy and sell strategy. Optimization is done by maximazing percentage of successful trades, taking into account number of trades.

In [ ]:
# imports
from scraper import stock_daily
from analyzer import Analyzer
from IPython.display import display
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import indicators as ind

## Define ```test_strategy()```


In [ ]:
# define parameters Series


In [ ]:
def test_strategy(ticker="None", ):
    # create empy series
    result = {"ticker": ticker, "trades": np.nan, "gtrades": np.nan, "profit[$]": np.nan, "profit[%]": np.nan, "hold[%]": np.nan}
    result = pd.Series(data=result)
    # scraping / loading data
    try:
        stock = stock_daily(ticker, save=False)
    except:
        print(ticker + ": Exception occured during data scraping, skipped.")
        return
    # creating analyzer instance
    trades = Analyzer(ticker=ticker, data=stock.data)
    
    # SETTINGS
    # stop loss
    sl = True
    slval = 0.04
    # profit taker
    pt = False
    ptval = 0.11
    # repeteated buys
    rb = False
    
    # STRATEGY DEFINITIONS
    # buy strategy
    # calculating VFI
    vfi = ind.vfi(stock.data, period=30, coef=0.2, vcoef=1.5)
    # calculating VFI histogram trend
    window = 5
    trend = vfi["histogram"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    trend = trend > 0.08
    # buy signals from stochastic oscilator
    bss = trades.mb_stoch(period=21, sk=3, sd=5, treshold=20, tcross="d")
    # finalize buy signals
    bsf = np.logical_and(trend, bss)
    # throw exception if buy signal is empty
    if bsf.sum() == 0:
        print(ticker + ": No buying signals generated, skipped.")
        return
    # sell strategy
    # sell after n days
    ss = np.roll(bsf, 10)
    ssf = np.repeat([ss,], repeats=np.sum(bsf), axis=0)
    # sort signals
    ss = trades.signalSorter(bsf,ssf,rb)
    
    # building final strategy
    trades.strategy(ss, stopLoss=sl, stopLossValue=slval, profitTaker=pt, profitTakerValue=ptval)
    # evaluating strategy
    score = trades.profit(capitalForEachTrade=250, comission=2)
    # summarizing the stock trades
    result["trades"] = score.shape[0]
    result["gtrades"] = score["Good trade?"].sum().astype("int")
    result["profit[$]"] = np.round(score["Profit[$]"].sum(),2)
    result["profit[%]"] = np.round(score["Profit[%]"].sum(),2)
    result["hold[%]"] = np.round(100*((trades.data["Close"].iloc[-1]-trades.data["Close"].iloc[0])/trades.data["Close"].iloc[0]),2)
    return result